In [7]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, mean_squared_log_error, median_absolute_error, r2_score
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from IPython.core.debugger import set_trace
import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [8]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 80} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [9]:
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=365):
    # flatten data
#     data = train.values
    data = train
#     set_trace()
    X, y = list(), list()
    in_start = 0
    # step over the entire history one time step at a time
    for _ in range(len(data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        # ensure we have enough data for this instance
        if out_end <= len(data):
            x_input = data[in_start:in_end, 0]
            x_input = x_input.reshape((len(x_input), 1))
            X.append(x_input)
            y.append(data[in_end:out_end, 0])
        # move along one time step
        in_start += 1
#     set_trace()
    return array(X), array(y)

In [10]:
# train the model
def build_model(train, n_input):
#     # prepare data
#     train_x, train_y = to_supervised(train, n_input)
#     # define parameters
#     verbose, epochs, batch_size = 1, 100, 256
#     n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
#     # define model
#     model = Sequential()
#     model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(n_outputs, activation='relu'))
#     model.compile(loss='mse', optimizer='adam')
#     # fit network
#     model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
#     return model
#######################################################################
    # prepare data
    train_x, train_y = to_supervised(train, n_input)
    # define parameters
    verbose, epochs, batch_size = 1, 20, 16
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    # reshape output into [samples, timesteps, features]
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
    # define model
    model = Sequential()
    model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(RepeatVector(n_outputs))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(100, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
    return model

In [11]:
def evaluate_model(train, test, n_input):
    test = test.dropna(axis=0)
    train = train.dropna(axis=0)
    scaler = MinMaxScaler(feature_range=(-1,1))
    train = scaler.fit_transform(train)
    # fit model
    model = build_model(train, n_input)
    input_data = train[-n_input:]
    input_data = input_data.reshape(1,n_input,1)
    output_data = model.predict(input_data, verbose=1)
    
    output_data = output_data[0]
#     set_trace()
    output_data = output_data.reshape(len(output_data),1)
    output_data = scaler.inverse_transform(output_data)
    true_data = test.values
    n = min(output_data.shape[0],true_data.shape[0])
    y_pred = output_data[:n]
    y_true = true_data[:n]
    mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
    print('MSE:', mse)
    evs = explained_variance_score(y_true=y_true,y_pred=y_pred)
    print('EVS:', evs)
    mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)
    print('MAE:', mae)
    msle = mean_squared_log_error(y_true=y_true,y_pred=y_pred)
    print('MSLE:', msle)
    meae = median_absolute_error(y_true=y_true,y_pred=y_pred)
    print('MEAE:', meae)
    r2 = r2_score(y_true=y_true,y_pred=y_pred)
    print('R2-Score:', r2)

In [12]:
train = read_csv('../data/Daily_Load_Data/train/coast.csv',index_col='Time')
test = read_csv('../data/Daily_Load_Data/test/coast.csv',index_col='Time')
n_input = 365
evaluate_model(train, test, n_input)

1/1 [==============================] - 1s 1s/step


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').